In [7]:
%pip install tiktoken
%pip install openai
%pip install textract
%pip install numpy
%pip install pandas


DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.
DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.
DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 23.3 will enforce this behaviou

# 라이브러리 임포트

In [214]:
import os
import requests
import numpy as np
import pandas as pd
from typing import Iterator
import tiktoken
import textract

# 논문 파일 리스트 구하기

In [281]:
import os

data_path = "data/paper"
pdf_files = [x for x in os.listdir(data_path)]

print(len(pdf_files))
print(pdf_files[:3])

20
['jkma-2023-66-1-50.pdf', 'jkma-2023-66-2-116.pdf', 'jkma-2023-66-3-191.pdf']


In [282]:
# pdf_files = pdf_files[:2] 

# 파일을 조각(chunk)로 쪼개고 임베딩해 두기

In [283]:
import openai
openai.api_key = "sk-..."

In [291]:
import tiktoken

TEXT_EMBEDDING_CHUNK_SIZE=int(4098/2.0)
TEXT_EMBEDDING_CHUNK_SIZE = 300
EMBEDDINGS_MODEL = "text-embedding-ada-002"
tokenizer = tiktoken.get_encoding("cl100k_base")

embedding_storage = []

# 긴 문자열을 최대 길이 chunk_size의 토큰들로 자르고, 토큰을 반환한다.
def chunks(text, chunk_size=TEXT_EMBEDDING_CHUNK_SIZE):
    """Yield successive chunk_size chunks from text."""
    tokens = tokenizer.encode(text)
    i = 0
    while i < len(tokens):
        # 토큰 길이의 0.5 ~ 1.5 사이에서 문장의 끝을 찾는다.
        j = min(i + int(1.5 * chunk_size), len(tokens))
        while j > i + int(0.5 * chunk_size):
            # 디코딩 해서 마침표 혹은 줄 바꿈인지 확인한다.
            chunk = tokenizer.decode(tokens[i:j])
            if chunk.endswith(".") or chunk.endswith("\n"):
                break
            j -= 1
        # 문장 끝을 못찾으면 chunk_size로 한다.
        if j == i + int(0.5 * chunk_size):
            j = min(i + chunk_size, len(tokens))
        yield tokens[i:j]
        i = j

def get_embeddings(text_array, engine=EMBEDDINGS_MODEL):
    # embedding_objects = openai.Embedding.create(text_array, model=EMBEDDINGS_MODEL)["data"]
    embedding_objects = openai.Engine(id=engine).embeddings(input=text_array)["data"]
    return [embedding_object["embedding"] for embedding_object in embedding_objects]

    
def create_embeddings_for_text(text):
    # 최대길이 TEXT_EMBEDDING_CHUNK_SIZE의 토큰 리스트로 변환. 쪼개진 개별을 chunk라 한다.
    token_chunks = list(chunks(text))
    # 각 토큰 리스트를 text 리스트로 변환
    text_chunks = [tokenizer.decode(chunk) for chunk in token_chunks]

    # 전체 chunk들의 임베딩을 구하고
    embedding_chunks = get_embeddings(text_chunks)
    return text_chunks, embedding_chunks


def handle_file_string(filename, file_content_string):
    
    # 파일 내용에서 라인 변경, 더블 스페이스, 세미 콜론을 삭제
    clean_file_content_string = " ".join(file_content_string.replace("\n", "; ").replace(";", " ").split())
        
    try:
        text_chunks, embedding_chunks = create_embeddings_for_text(clean_file_content_string)
        print(f"[INFO] Embedded into {len(text_chunks)} chunks")
    except Exception as e:
        print("[handle_file_string] Error creating embedding: {}".format(e))

    for i, (text_chunk, embedding_chunk) in enumerate(zip(text_chunks, embedding_chunks)):
        embedding_storage.append({"embedding":embedding_chunk, "filename": filename, "chunk_index":i, "text":text_chunk })


In [292]:

# Process each PDF file and prepare for embedding
for i, pdf_file in enumerate(pdf_files):
    
    pdf_path = os.path.join(data_path, pdf_file)
    print(f"[INFO] {i}/{len(pdf_files)}. processing paper : {pdf_path}\n")
    
    text = textract.process(pdf_path, method='pdfminer')
    print(text.decode("utf-8")[0:200])
    print()

    handle_file_string(pdf_file, text.decode("utf-8"))
    print()
    print("============================================================")


[INFO] 0/20. processing paper : data/paper/jkma-2023-66-1-50.pdf

SPECIAL CONTRIBUTION 
J Korean Med Assoc 2023 January; 66(1):50-59

pISSN 1975-8456 / eISSN 2093-5951

https://doi.org/10.5124/jkma.2023.66.1.50

만성 코로나19 증후군 시대를 위한  
만성 코로나19 증후군 시대를 위한  
보건의료 대응 방안

[INFO] Embedded into 49 chunks

[INFO] 1/20. processing paper : data/paper/jkma-2023-66-2-116.pdf

FOCUSED ISSUE OF THIS MONTH 
J Korean Med Assoc 2023 February; 66(2):116-122

pISSN 1975-8456 / eISSN 2093-5951

https://doi.org/10.5124/jkma.2023.66.2.116

면역항암치료의 부작용 관리: 호흡계와 신경계를  
면역항암치료의 부작용 관리:

[INFO] Embedded into 28 chunks

[INFO] 2/20. processing paper : data/paper/jkma-2023-66-3-191.pdf

 의학강좌

CONTINUING EDUCATION COLUMN 
J Korean Med Assoc 2023 March; 66(3):191-197

pISSN 1975-8456 / eISSN 2093-5951

https://doi.org/10.5124/jkma.2023.66.3.191

선천성 갑상샘저하증의 진단과 치료
선천성 갑상샘저하증의 진단과 치료


[INFO] Embedded into 25 chunks

[INFO] 3/20. processing paper : data/paper/jkma-2023-66-3-200.pdf

ORIGINAL ARTICLE 
J Korean Med 

In [293]:
from pprint import pprint

print(len(embedding_storage))
print(embedding_storage[0]["embedding"])
print(len(embedding_storage[0]["embedding"]))
print(embedding_storage[0]["filename"])
print(embedding_storage[0]["chunk_index"])
print(embedding_storage[0]["text"])

675
[-0.008223154, -0.010025766, 0.02427788, -0.033648767, -0.0050837724, 0.0085944785, -0.014407397, 0.0030549893, -0.02606024, 0.025628153, -0.009701701, 0.02859875, -0.01625727, 0.027680567, -0.020024527, -0.020024527, 0.038617764, -0.012787071, 0.0064678006, -0.023751276, -0.012793822, -0.0022009425, -0.031947423, -0.0070484173, 0.0054112133, 0.018242167, -0.0037875115, -0.025628153, -0.007088925, 3.0301992e-05, 0.009850231, -0.015433603, -0.012321227, 0.010079777, 0.008081375, 0.0048913583, 0.009384387, -0.01736449, 0.020416105, -0.017283475, 0.021590842, 0.017040426, -0.0045571662, -0.001797549, -0.007169942, -0.004813718, 0.015082533, -0.031245282, -0.014798976, 0.023589244, 0.013084131, 0.0014928939, 0.0036423574, 0.0032237733, -0.0049825017, -0.013090882, 0.01198366, 0.01810714, 0.0072577093, -0.02049712, 0.010910193, -0.015393095, -0.020956215, 0.0022262603, -0.012942352, -0.005218799, -0.023791784, 0.013468958, -0.01651382, 0.024250876, 0.02322467, 0.02930089, 0.009181846, -

# 가장 가까운 조각(chunk) 구하기

In [304]:
def dist(a, b):
    return np.linalg.norm(np.array(a)-np.array(b))

def find_closest_index(embedding_storage, embedded_query):
    return min(range(len(embedding_storage)), key = lambda i: dist(embedding_storage[i]["embedding"], embedded_query))

def find_closest(embedding_storage, embedded_query):
    closest_index = find_closest_index(embedding_storage, embedded_query)
    return embedding_storage[closest_index]

def find_closest_chunk(embedding_storage, query):
    embedded_query = get_embeddings(query)[0]
    closest_index = find_closest_index(embedding_storage, embedded_query)
    return embedding_storage[closest_index]



def find_closest_n_index(embedding_storage, embedded_query, n=3):
    return sorted(range(len(embedding_storage)), key = lambda i: dist(embedding_storage[i]["embedding"], embedded_query))[:n]
    return min(range(len(embedding_storage)), key = lambda i: dist(embedding_storage[i]["embedding"], embedded_query))


def find_closest_n_chunk(embedding_storage, query, n=3):
    embedded_query = get_embeddings(query)[0]
    closest_indexes = find_closest_n_index(embedding_storage, embedded_query, n)
    return [embedding_storage[i] for i in closest_indexes]


In [315]:
question = "코로나19 증후군의 증상은?"
queryed = find_closest_n_chunk(embedding_storage, question, n=1)[0]

In [316]:
print(queryed["filename"])
print(queryed["chunk_index"])
print(queryed["text"])


jkma-2023-66-3-200.pdf
4
 질병관리청에서는 확진자 더 낮아졌을 것으로 기대할 수 있다. 이 연구에서는 군대에 의 10-20% 정도는 초기 질병 회복 후에도 피로, 호흡곤란 서 자체적으로 실시한 기존 코로나19 확진자에 대한 지속적 및 인지기능 장애, 기침, 미각·후각 이상, 우울증과 불안, 인 상담 및 자가 기입식 설문조사 결과자료를 분석하여 군대 수면장애 등 다양한 중장기적 영향을 경험한다고 설명하고 의 코로나19 후유증(post-acute COVID-19 syndrome) 발 있다[1]. 또한 2020년 2-3월 대구에서 발생한 코로나19 확 생률을 확인하고 연령과 성별과 같은 인구학적 요인, 계급이 진자 900여 명을 대상으로 조사한 결과 코로나19 환자의 연 나 격리시설의 종류, 어느 우세종이 우세한 시기에 코로나19 령, 성별, 코로나19 중증도는 지속적인 신경정신과적 증상 에 확진되었는지 등의 요인과 코로나19 후유증 발생 여부 간 발생과 상관관계가 있는 것으로 나타났다[2]. 의 상관관계를 확인하고자 한다.


# 구한 조각 내용으로 문의하기

In [317]:
prompt = f"""You will be provided with text delimited by triple quotes and question. answer the question based on the provided text

Text:
```{queryed["text"]}```

Question:
```{question}```
"""


response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are helpful QnA engine."},
        {"role": "user", "content": prompt},
    ],
    temperature=0,
)

print(response['choices'][0]['message']['content'])

코로나19 증후군의 증상은 피로, 호흡곤란, 기침, 미각·후각 이상, 우울증과 불안, 수면장애 등 다양한 중장기적 영향을 경험할 수 있다고 설명되었습니다.
